In [2]:
#importing important libaries 
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [42]:
#Creating empty lists for the information that will be stored
Year = []
Brand = []
Model = []
State = []
Milage = []
Lot = []
Price = []
Body = []

In [44]:
#A list of different body types which is used in the function
body_types = ["SUV", "Sedan", "Convertible", "Wagon", "Hatchback", "Pickup", "Van", "Utility", "Cruiser"]

In [118]:
#Creating the web scraping function
def scrape_cars():
    cars = soup.find_all("div", class_ = "ae-flex-w-100 ae-flex-w-md-70 text-truncate")
    for car in cars:
        car_name = car.find("div", class_ = "col text-truncate").a.text
        milage = car.find_all("div", class_ = "col-6 col-lg-7 ae-text-dark-blue text-truncate")[1].text
        location = car.find_all("div", class_ = "col-6 col-lg-7 ae-text-dark-blue text-truncate")[5].text
        lot = car.find_all("div", class_ = "col-6 col-lg-7 ae-text-dark-blue text-truncate")[0].text
        year = car_name.split(" ")[0]
        brand = car_name.split(" ")[1]
        model = car_name.split(" ")[2]
        state = location.split(",")[0]
        price_tab = car.find("div", class_ = "row align-items-center no-gutters pt-1 pt-lg-2 px-2 px-md-3")
        price_1 = price_tab.find("div", class_ = "col-auto ae-fs-24 ae-fw-700 ae-text-dark-blue")
        if price_1 is not None:
            price_2 = price_1.text.replace(",", "")
            price = int(price_2[1:].strip())
            if "buy" in price_tab.text.lower():
                Year.append(year)
                Brand.append(brand)
                Model.append(model)
                State.append(state)
                Milage.append(milage)
                Lot.append(lot)
                Price.append(price)
                match_found = False
                for types in body_types:
                    if types.lower() in car_name.lower():
                        Body.append(types)
                        match_found = True
                        break
                if not match_found:
                    Body.append("Unknown")

In [120]:
#Calling the web scraping function for the first 850 pages
for i in range(1, 851):
    url = "https://www.auctionexport.com/en/cars/clean/page/"
    page = requests.get(url + str(i))
    soup = BeautifulSoup(page.text, "html")
    scrape_cars()

In [128]:
len(Price)

5013

In [130]:
#Turning the lists into a pandas dataframe
data = {
    "Year": Year,
    "Brand": Brand,
    "Model": Model,
    "State": State,
    "Milage": Milage,
    "Lot": Lot,
    "Price": Price
}
df = pd.DataFrame(data)

In [132]:
df

,Year,Brand,Model,State,Milage,Lot,Price
0,2014,JEEP,GRAND,Michigan,141611,206958611,3250
1,2014,HONDA,ACCORD,Michigan,262803,206958621,1425
2,1999,HONDA,CR-V,Michigan,214984,206958645,425
3,2011,HYUNDAI,SONATA,Michigan,171272,206958649,900
4,2001,HONDA,CR-V,Michigan,218334,206958650,500
...,...,...,...,...,...,...,...
5008,2012,MERCEDES-BENZ,M-CLASS,Nevada,125917,206920207,4325
5009,1995,CHEVROLET,BLAZER,Oklahoma,129301,206846793,325
5010,2014,HYUNDAI,ELANTRA,California,1,206887337,950
5011,1999,LEXUS,RX,Montana,233343,206847510,475


In [134]:
#Exporting the table into a csv file
df.to_csv("Auction_Export")